In [ ]:
import sqlite3, csv

In [ ]:
csv_files = ['Menu.csv', 'MenuPage.csv', 'MenuItem.csv', 'Dish.csv']

# Read the CSV file as a database
def read_csvs_as_db(csv_files):
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    for csv_file in csv_files:
        table_name = csv_file.split('/')[-1].split('.')[0]
        print(f'Processing {csv_file} into table {table_name}')
        with open(csv_file, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            headers = next(reader)
            # Add type affinity for better numeric handling
            columns = ', '.join([f'"{h}"' for h in headers])
            cursor.execute(f'CREATE TABLE IF NOT EXISTS {table_name} ({columns})')
            for row in reader:
                cursor.execute(f'INSERT INTO {table_name} VALUES ({", ".join(["?"] * len(row))})', row)
    conn.commit()
    return conn

In [ ]:
conn = read_csvs_as_db(csv_files)
cursor = conn.cursor()

# Primary Key Uniqueness

In [ ]:
cursor.execute('''
    SELECT *
        FROM Menu
        GROUP BY id
        HAVING COUNT(*) > 1;
''')

result = cursor.fetchall()
if not result:
    print("No duplicate entries found in Menu.")
else:
    print("Duplicate entries found in Menu:")
    print(result)


cursor.execute('''
    SELECT *
        FROM MenuItem 
        GROUP BY id
        HAVING COUNT(*) > 1;
''')

result = cursor.fetchall()
if not result:
    print("No duplicate entries found in MenuItem.")
else:
    print("Duplicate entries found in MenuItem:")
    print(result)

cursor.execute('''
    SELECT *
        FROM MenuPage
        GROUP BY id
        HAVING COUNT(*) > 1;
''')

result = cursor.fetchall()
if not result:
    print("No duplicate entries found in MenuPage.")
else:
    print("Duplicate entries found in MenuPage:")
    print(result)

cursor.execute('''
    SELECT *
        FROM Dish
        GROUP BY id
        HAVING COUNT(*) > 1;
''')

result = cursor.fetchall()
if not result:
    print("No duplicate entries found in Dish.")
else:
    print("Duplicate entries found in Dish:")
    print(result)

# Foreign Key Validity

In [ ]:
cursor.execute('''
    SELECT *
        FROM MenuPage
        WHERE menu_id NOT IN (SELECT id FROM Menu);
''')
result = cursor.fetchall()
if not result:
    print("All MenuPage entries are linked to a valid Menu.")
else:
    print("MenuPage entries linked to an invalid Menu:")
    print(result)

cursor.execute('''
    SELECT *
    FROM MenuItem
    WHERE menu_page_id NOT IN (SELECT id FROM MenuPage);
''')
result = cursor.fetchall()
if not result:
    print("All MenuItem entries are linked to a valid MenuPage.")
else:
    print("MenuItem entries linked to an invalid MenuPage:")
    print(result)

cursor.execute('''
    SELECT *
    FROM MenuItem
    WHERE dish_id NOT IN (SELECT id FROM Dish);
''')
result = cursor.fetchall()
if not result:
    print("All MenuItem entries are linked to a valid Dish.")
else:
    print("MenuItem entries linked to an invalid Dish:")
    print(result)

# Value Ranges

### Menu

In [ ]:
cursor.execute('''
    SELECT *
        FROM Menu
        WHERE page_count < 0 or dish_count < 0;
''')

result = cursor.fetchall()
if not result:
    print("No negative values found in Menu.")
else:
    print("Negative values found in Menu:")
    print(result)

cursor.execute('''
    SELECT * 
        FROM Menu
        JOIN MenuPage
        ON Menu.id = MenuPage.menu_id
        GROUP BY menu_id HAVING COUNT(*) < Menu.page_count;
''')

result = cursor.fetchall()
if not result:
    print("All Menu entries have correct page counts.")
else:
    print("Menu entries with incorrect page counts:")
    print(result)

### MenuItem

In [ ]:
cursor.execute('''
    SELECT *
        FROM MenuItem
        WHERE created_at > updated_at;
''')

result = cursor.fetchall()
if not result:
    print("No MenuItem entries with created_at later than updated_at.")
else:
    print("MenuItem entries with created_at later than updated_at:")
    print(result)

cursor.execute('''
    SELECT *
    FROM MenuItem
    WHERE price IS NULL OR price = 0;
''')
result = cursor.fetchall()
if not result:
    print("No MenuItem entries with null or zero price.")
else:
    print("MenuItem entries with null or zero price:")
    print(result)

cursor.execute('''
    SELECT *
        FROM MenuItem
        WHERE price < 0;
''')

result = cursor.fetchall()
if not result:
    print("No MenuItem entries with created_at later than updated_at.")
else:
    print("MenuItem entries with created_at later than updated_at:")
    print(result)

cursor.execute('''
    SELECT MenuItem.*, Dish.highest_price, Dish.lowest_price
        FROM MenuItem
        JOIN Dish ON MenuItem.dish_id = Dish.id
        WHERE MenuItem.price > Dish.highest_price 
            OR MenuItem.price < Dish.lowest_price;
''')

result = cursor.fetchall()
if not result:
    print("No MenuItem entries with price outside Dish price range.")
else:
    print("MenuItem entries with price outside Dish price range:")
    print(result)

### MenuPage

In [ ]:
cursor.execute('''
    SELECT *
        FROM MenuPage
        WHERE page_number < 0 OR full_height < 0 OR full_width < 0;
''')

result = cursor.fetchall()
if not result:
    print("No MenuPage entries with negative dimensions.")
else:
    print("MenuPage entries with negative dimensions:")
    print(result)

### Dish

In [ ]:
cursor.execute('''
    SELECT *
        FROM Dish
        WHERE first_appeared > last_appeared;
''')

result = cursor.fetchall()
if not result:
    print("No Dish entries with first_appeared later than last_appeared.")
else:
    print("Dish entries with first_appeared later than last_appeared:")
    print(result)

cursor.execute('''
    SELECT *
        FROM Dish
        WHERE lowest_price > highest_price;
''')

result = cursor.fetchall()
if not result:
    print("No Dish entries with lowest_price greater than highest_price.")
else:  
    print("Dish entries with lowest_price greater than highest_price:")
    print(result)

cursor.execute('''
    SELECT *
        FROM Dish
        WHERE menus_appeared < 0 
            OR times_appeared < 0 
            OR first_appeared < 0 
            OR last_appeared < 0
            OR lowest_price < 0 
            OR highest_price < 0;
''')

result = cursor.fetchall()
if not result:
    print("No Dish entries with negative values in critical fields.")
else:
    print("Dish entries with negative values in critical fields:")
    print(result)

# Use Case Checks

### Clustering Success Check

In [ ]:
cursor.execute('''
    SELECT * FROM Dish
        WHERE name LIKE '%coffee%'
            AND name NOT LIKE 'coffee';
''')

result = cursor.fetchall()
if not result:
    print("No Dish entries with name containing 'coffee' but not exactly 'coffee'.")
else:
    print("Dish entries with name containing 'coffee' but not exactly 'coffee':")
    print(result)

cursor.execute('''
    SELECT 
        (CAST(strftime('%Y', Menu.Date) AS INTEGER) / 10) * 10 AS Decade,
        AVG(MenuItem.price) AS Avg_Coffee_Price
    FROM Menu
    JOIN MenuPage ON Menu.id = MenuPage.menu_id
    JOIN MenuItem ON MenuItem.menu_page_id = MenuPage.id
    JOIN Dish_clustered ON Dish_clustered.id = MenuItem.dish_id
    WHERE 
        Dish_clustered.name = 'coffee'
        AND Menu.currency = 'Dollars'
        AND Menu.currency_symbol = '$'
        AND CAST(strftime('%Y', Menu.Date) AS INTEGER) BETWEEN 1900 AND 2000
        AND MenuItem.price IS NOT NULL   
    GROUP BY Decade
    ORDER BY Decade;
''')

print("Average Coffee Price by Decade:")
print(cursor.fetchall())

